In [1]:
# Install the latest stable versions compatible with each other
!pip install -U streamlit
!pip install -U pyngrok
!pip install -U langchain==1.2.7
!pip install -U langchain_classic
!pip install -U langchain_community
!pip install -U langchain-pinecone
!pip install -U sentence-transformers
!pip install -U pypdf
!pip install langchain-groq
!pip install langchain-text-splitters
!pip install cassio
!pip install langchain_astradb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.1/329.1 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 495.8/495.8 kB 43.5 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 1.2.7
    Uninstalling langchain-core-1.2.7:
      Successfully uninstalled langchain-core-1.2.7
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.4/374.4 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 333.5/333.5 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 99.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.1/331.1 kB 33.6 MB/s eta 0:00:00


In [ ]:
!pip install sentence-transformers
!pip install langchain_huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.3/566.3 kB 35.8 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface_hub 1.3.4
    Uninstalling huggingface_hub-1.3.4:
      Successfully uninstalled huggingface_hub-1.3.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 5.0.0 requires huggingface-hub<2.0,>=1.3.0, but you have huggingface-hub 0.36.1 which is incompatible.


In [ ]:
from google.colab import userdata
import os

# Fetch secrets from Colab's secret manager
GROQ_API_KEY = userdata.get("GROQ_API_KEY")

# Set them as environment variables for libraries that read them
import os
os.environ["GROQ_API_KEY"] = GROQ_API_KEY

In [ ]:
import os
from pyngrok import ngrok

# --- STEP 1: DEFINE STREAMLIT APP ---
app_code = """
import streamlit as st
import bs4
import time
import cassio
from langchain_groq import ChatGroq
from langchain_community.document_loaders import WebBaseLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores.cassandra import Cassandra
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_classic.chains import create_retrieval_chain


st.set_page_config(page_title="AstraDB RAG Demo", layout="wide")

# Sidebar for API Keys
with st.sidebar:
    st.title("Configuration")
    groq_api_key = st.text_input("Groq API Key", type="password")
    astra_db_application_token = os.getenv("ASTRA_DB_APPLICATION_TOKEN")
    astra_db_id = os.getenv("ASTRA_DB_ID")"

st.title("🚀 AstraDB + Groq RAG")

if not groq_api_key:
    st.info("Please enter your Groq API Key in the sidebar to continue.")
    st.stop()

# Initialize Vector Store
if 'vector_store' not in st.session_state:
    with st.spinner("Initializing Astra DB..."):
        cassio.init(token=astra_token, database_id=astra_id)
        loader = WebBaseLoader(
            web_paths=("https://lilianweng.github.io",),
            bs_kwargs=dict(parse_only=bs4.SoupStrainer(class_=("post-title","post-content","post-header")))
        )
        docs = loader.load()
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
        final_docs = text_splitter.split_documents(docs)

        embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
        st.session_state.vector_store = Cassandra(
            embedding=embeddings,
            table_name="qa_mini_demo_streamlit",
            session=None,
            keyspace=None
        )
        st.session_state.vector_store.add_documents(final_docs)

# Chain Setup
llm = ChatGroq(groq_api_key=groq_api_key, model_name="llama-3.3-70b-versatile")
prompt = ChatPromptTemplate.from_template(\"\"\"
Answer the following question based only on the provided context.
<context>{context}</context>
Question: {input}\"\"\")

document_chain = create_stuff_documents_chain(llm, prompt)
retriever = st.session_state.vector_store.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

user_prompt = st.text_input("Ask about AI Agents:")

if user_prompt:
    start = time.process_time()
    response = retrieval_chain.invoke({"input": user_prompt})
    st.markdown(f"### Answer\\n{response['answer']}")
    st.caption(f"Response time: {time.process_time() - start:.2f}s")
"""

# --- STEP 2: RUN IN COLAB ---
with open("app.py", "w") as f:
    f.write(app_code)

# Auth and Tunnel
import getpass
print("Enter your ngrok Auth Token (from https://dashboard.ngrok.com):")
NGROK_AUTH_TOKEN = getpass.getpass()
!ngrok config add-authtoken {NGROK_AUTH_TOKEN}

!kill $(lsof -t -i:8501) 2>/dev/null
public_url = ngrok.connect(8501)
print(f"\\nSTREMLIT APP URL: {public_url.public_url}")

!streamlit run app.py &

Enter your ngrok Auth Token (from https://dashboard.ngrok.com):
··········
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
\nSTREMLIT APP URL: https://unrelievable-danette-jointedly.ngrok-free.dev





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.240.143.144:8501

  Stopping...


In [ ]:
# 1. Kill any existing processes
!pkill streamlit
!pkill ngrok

# 2. Start Streamlit in the background and wait 10 seconds
# We use --server.address=0.0.0.0 to ensure it's accessible
!nohup streamlit run app.py --server.port 8501 --server.address 0.0.0.0 &>/content/logs.txt &
import time
time.sleep(10)

# 3. Create a fresh tunnel
from pyngrok import ngrok
# Replace with your actual token if not already set
# ngrok.set_auth_token("")

# Disconnect existing tunnels to avoid "Too many sessions" error
tunnels = ngrok.get_tunnels()
for t in tunnels:
    ngrok.disconnect(t.public_url)

public_url = ngrok.connect(8501)
print(f"New App URL: {public_url.public_url}")

New App URL: https://unrelievable-danette-jointedly.ngrok-free.dev
